In [ ]:
using JuMP
import Ipopt
using Distributions
using Dates
using JLD2
include("utilities.jl")
folder = pwd()

In [ ]:
actions_states = [2 3; 3 4; 3 5]
D = [[[3], [2 1], [1 1 1]], [[4], [3 1], [2 2], [2 1 1], [1 1 1 1]], [[5], [3 2], [4 1], [3,1,1], [2,2,1], [2,1,1,1], [1,1,1,1,1]]]
iterations = 20

In [ ]:
for i in 1:size(actions_states)[1]
    (nA, nS) =  actions_states[i,:]
    for j in 1:length(D[i])
        dₒ = D[i][j]
        nO = length(dₒ)
        println("nA = ", nA, ", nS = ", nS, ", dₒ = ", dₒ)
        for it in 1:iterations

            γ = 0.9
            Mβ = load_object(string(folder,"/Data/beta_", nA, "_", nS, "_", nO, "_", j, "_", it))
            Mα = load_object(string(folder,"/Data/alpha_", nA, "_", nS, "_", nO, "_", j, "_", it))
            r = load_object(string(folder,"/Data/r_", nA, "_", nS, "_", nO, "_", j, "_", it))
            μ = load_object(string(folder,"/Data/mu_", nA, "_", nS, "_", nO, "_", j, "_", it))

            ### Use Ipopt to optimize
            tIpopt = @elapsed ηIpopt = IpoptSolve(Mα, Mβ, r, γ, μ);
            optIpopt = transpose(reshape(r, nS*nA))*reshape(ηIpopt, nS*nA)
            
            ### Print Data
            extraInfoIpopt = ("Ipopt", tIpopt, optIpopt)
            infoFile = string(folder, "/Results/Ipopt/Ipopt_info_", nA, "_", nS, "_", nO, "_", j, "_", it, ".txt")
            open(infoFile, "w") do f
                println(f, extraInfoIpopt)
            end
        end
    end
end
